## Problem 1

In [2]:
import numpy as np

In [9]:
A = np.array([[ 1, 2, 3, 4],
              [ 2, 4,-4, 8],
              [-5, 4, 1, 5],
              [ 5, 0,-3,-7]])
eig_vals, eig_vecs = np.linalg.eig(A.transpose()@A)
max_eig_val_index = np.argmax(np.abs(eig_vals))
print(f"L2 norm of A is: {np.sqrt(eig_vals[max_eig_val_index])}")

L2 norm of A is: 13.858100376465332


In [10]:
eig_vecs

array([[-0.29618621,  0.7044906 , -0.64207608, -0.060869  ],
       [ 0.35616716,  0.26659364,  0.21072852, -0.87044028],
       [ 0.06730298, -0.63349701, -0.69440426, -0.33459612],
       [ 0.88367923,  0.17692476, -0.24725395,  0.35591308]])

In [11]:
x = eig_vecs.transpose()[max_eig_val_index]
x

array([-0.29618621,  0.35616716,  0.06730298,  0.88367923])

In [12]:
np.linalg.norm(A@x, ord=2) / np.linalg.norm(x, ord=2)

13.858100376465325

In [13]:
x_sol = np.array([-0.296, 0.356, 0.067, 0.884])

In [14]:
np.linalg.norm(A@x_sol)/np.linalg.norm(x_sol)

13.858099042365327

# Problem 4

## Section A

In [2]:
a = np.array([[2,  1, 2],
              [1, -2, 1],
              [1,  2, 3],
              [1,  1, 1]])
b = np.array([6, 1, 5, 2])

In [3]:
at_a = a.transpose()@a
at_b = a.transpose()@b

In [4]:
l = np.linalg.cholesky(at_a)
l

array([[2.64575131, 0.        , 0.        ],
       [1.13389342, 2.9519969 , 0.        ],
       [3.40168026, 1.06465462, 1.51495279]])

In [5]:
def bwd_sub(U, b):
    n = np.size(b)
    x = np.zeros(n, dtype=np.float64)
    for i in reversed(range(0, n)):
        sum = 0
        for j in range(i, n):
            sum += U[i, j] * x[j]
        x[i] = (1/U[i, i]) * (b[i] - sum)
    return x

def fwd_sub(L, b):
    n = np.size(b)
    x = np.zeros(n, dtype=np.float64)
    for i in range(0, n):
        sum = 0
        for j in range(0, i):
            sum += L[i, j] * x[j]
        x[i] = (1/L[i, i]) * (b[i] - sum)
    return x

In [6]:
y = fwd_sub(l, at_b)
x = bwd_sub(l.transpose(), y)
x

array([1.7, 0.6, 0.7])

In [7]:
x - np.linalg.solve(at_a, at_b)

array([-1.55431223e-15, -5.55111512e-16,  1.33226763e-15])

## Section B - QR

In [8]:
q, r = np.linalg.qr(a)

In [9]:
q

array([[-0.75592895,  0.04839339,  0.41120147],
       [-0.37796447, -0.82268766, -0.38955929],
       [-0.37796447,  0.53232731, -0.7574764 ],
       [-0.37796447,  0.19357357,  0.32463274]])

In [10]:
r

array([[-2.64575131, -1.13389342, -3.40168026],
       [ 0.        ,  2.9519969 ,  1.06465462],
       [ 0.        ,  0.        , -1.51495279]])

In [11]:
x = bwd_sub(r, q.transpose()@b)
x

array([1.7, 0.6, 0.7])

## Section B - SVD

In [12]:
u, sigma, v_t = np.linalg.svd(a, full_matrices=False)

In [13]:
y = (u.transpose()@b)/sigma

In [14]:
x = v_t.transpose()@y
x

array([1.7, 0.6, 0.7])

## Section C

In [15]:
r = a@x - b
r

array([-6.00000000e-01,  2.00000000e-01, -1.77635684e-15,  1.00000000e+00])

In [16]:
a.transpose()@r

array([-9.32587341e-15, -9.76996262e-15, -1.28785871e-14])

## Section D

In [101]:
w = np.array([[900, 0, 0, 0],
              [  0, 1, 0, 0],
              [  0, 0, 1, 0],
              [  0, 0, 0, 1]])

In [102]:
l = np.linalg.cholesky(a.transpose()@w@a)

In [103]:
y = fwd_sub(l, a.transpose()@w@b)
x = bwd_sub(l.transpose(), y)
x

array([2.1723696 , 0.69216968, 0.48109701])

In [104]:
a@x - b

array([-8.97090862e-04,  2.69127259e-01, -8.92619312e-13,  1.34563629e+00])

# Problem 5

In [17]:
def gram_schmidt_QR(A):
    m, n = A.shape
    R = np.zeros((n,n))
    Q = np.zeros((m,n))

    R[0,0] = np.linalg.norm(A[:,0], ord=2)
    Q[:,0] = A[:,0]/R[0,0]

    for i in range(1,n):
        Q[:,i] = A[:,i]
        for j in range(0,i):
            R[j,i] = Q[:,j] @ A[:,i]
            Q[:,i] = Q[:,i] - R[j,i] * Q[:,j]
        R[i,i] = np.linalg.norm(Q[:,i], ord=2)
        Q[:,i] = Q[:,i] / R[i,i]
    
    return Q,R

def gram_schmidt_QR_modified(A):
    m, n = A.shape
    R = np.zeros((n,n))
    Q = np.zeros((m,n))

    R[0,0] = np.linalg.norm(A[:,0], ord=2)
    Q[:,0] = A[:,0]/R[0,0]

    for i in range(1,n):
        Q[:,i] = A[:,i]
        for j in range(0,i):
            R[j,i] = Q[:,j] @ Q[:,i]
            Q[:,i] = Q[:,i] - R[j,i] * Q[:,j]
        R[i,i] = np.linalg.norm(Q[:,i], ord=2)
        Q[:,i] = Q[:,i] / R[i,i]
    
    return Q,R

### For eps = 1

In [21]:
eps = 1
A = np.array([[1,   1,   1],
              [eps, 0,   0],
              [0,   eps, 0],
              [0,   0,   eps]])

In [22]:
q, r = gram_schmidt_QR(A)
print("Original Version:")
print(q)
print(r)

Original Version:
[[ 0.70710678  0.40824829  0.28867513]
 [ 0.70710678 -0.40824829 -0.28867513]
 [ 0.          0.81649658 -0.28867513]
 [ 0.          0.          0.8660254 ]]
[[1.41421356 0.70710678 0.70710678]
 [0.         1.22474487 0.40824829]
 [0.         0.         1.15470054]]


In [23]:
q, r = gram_schmidt_QR_modified(A)
print("Modified Version:")
print(q)
print(r)

Modified Version:
[[ 0.70710678  0.40824829  0.28867513]
 [ 0.70710678 -0.40824829 -0.28867513]
 [ 0.          0.81649658 -0.28867513]
 [ 0.          0.          0.8660254 ]]
[[1.41421356 0.70710678 0.70710678]
 [0.         1.22474487 0.40824829]
 [0.         0.         1.15470054]]


In [24]:
q, r = gram_schmidt_QR(A)
print("Original Version:")
print(f"Norm of (Q^T)Q - I: {np.linalg.norm(q.transpose()@q - np.eye(3), ord='fro')}")

Original Version:
Norm of (Q^T)Q - I: 4.777941758212735e-16


In [25]:
q, r = gram_schmidt_QR_modified(A)
print("Modified Version:")
print(f"Norm of (Q^T)Q - I: {np.linalg.norm(q.transpose()@q - np.eye(3), ord='fro')}")

Modified Version:
Norm of (Q^T)Q - I: 4.987305196443834e-16


### For eps = 1e-10

In [26]:
eps = 1e-10
A = np.array([[1,   1,   1],
              [eps, 0,   0],
              [0,   eps, 0],
              [0,   0,   eps]])

In [27]:
q, r = gram_schmidt_QR(A)
print("Original Version:")
print(q)
print(r)

Original Version:
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.00000000e-10 -7.07106781e-01 -7.07106781e-01]
 [ 0.00000000e+00  7.07106781e-01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  7.07106781e-01]]
[[1.00000000e+00 1.00000000e+00 1.00000000e+00]
 [0.00000000e+00 1.41421356e-10 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.41421356e-10]]


In [28]:
q, r = gram_schmidt_QR_modified(A)
print("Modified Version:")
print(q)
print(r)

Modified Version:
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.00000000e-10 -7.07106781e-01 -4.08248290e-01]
 [ 0.00000000e+00  7.07106781e-01 -4.08248290e-01]
 [ 0.00000000e+00  0.00000000e+00  8.16496581e-01]]
[[1.00000000e+00 1.00000000e+00 1.00000000e+00]
 [0.00000000e+00 1.41421356e-10 7.07106781e-11]
 [0.00000000e+00 0.00000000e+00 1.22474487e-10]]


In [29]:
q, r = gram_schmidt_QR(A)
print("Original Version:")
print(f"Norm of (Q^T)Q - I: {np.linalg.norm(q.transpose()@q - np.eye(3), ord='fro')}")

Original Version:
Norm of (Q^T)Q - I: 0.7071067811865477


In [30]:
q, r = gram_schmidt_QR_modified(A)
print("Modified Version:")
print(f"Norm of (Q^T)Q - I: {np.linalg.norm(q.transpose()@q - np.eye(3), ord='fro')}")

Modified Version:
Norm of (Q^T)Q - I: 1.1547005383855975e-10
